Learn 2 Run a Power Network
======
A Reinforcement Learning Challenge
-------

In [ ]:
%load_ext autoreload
import os
import sys
import grid2op
import matplotlib.pyplot as plt

# This is a small histogramming class -- see https://github.com/brendlin/matplotlibHistos
sys.path.insert(0, "../")
from matplotlibHistos.Histo import Histo

Let's get to know the system using the `rte_case14_realistic` environment:

In [ ]:
env = grid2op.make(test=True)

In [ ]:
from grid2op.PlotGrid import PlotMatplot
plot_helper = PlotMatplot(env.observation_space)
_ = plot_helper.plot_info(line_values=[el for el in range(env.n_line)])

How is the grid topology represented in the code?
------

In [ ]:
print('dim topo: env.dim_topo (2*n_line + n_load + n_gen): {:d} ({:d})'.format(env.dim_topo,env.n_line*2 + env.n_load + env.n_gen))
print('Number of lines, env.n_line:',env.n_line)
print('Number of loads, env.n_load:',env.n_load)
print('Number of loads, env.n_gen:',env.n_gen)
print('Number of connections to each sub env.sub_info: ',env.sub_info)

In [ ]:
print('Line origin    to subid env.line_or_to_subid:',env.line_or_to_subid)
print('Line extremity to subid env.line_ex_to_subid:',env.line_ex_to_subid)
print('Generator      to subid env.gen_to_subid    :',env.gen_to_subid)
print('Load           to subid env.load_to_subid   :',env.load_to_subid)
print()
print('Load-to-sub           position env.load_to_sub_pos   : ',env.load_to_sub_pos)
print('Generator-to-sub      position env.gen_to_sub_pos    : ',env.gen_to_sub_pos)
print('Line origin-to-sub    position env.line_or_to_sub_pos: ',env.line_or_to_sub_pos)
print('Line extremity-to-sub position env.line_ex_to_sub_pos: ',env.line_ex_to_sub_pos)

print('Topo vec (len {:d}): '.format(len(env.backend.get_topo_vect())),list(a for a in env.backend.get_topo_vect()))
#env.load_pos_topo_vect
#env.topo_vect
print()
print('env.grid_objects_types:\n',env.grid_objects_types[:10])
print('This will tell you the ID of the object connected to which bus. The order is [sub,load,gen,origin,extremity].')

Adjacency Matrix Representation of the Grid, and Connectivity checks (no buses):
--------
We will represent the grid using the buses and gens/loads as vertices, and the lines will be represented by an item a_{ij} of this matrix. See below:

In [ ]:
import numpy as np

# This helper module is mine:
import TopologyHelpers

In [ ]:
adjacency_matrix = TopologyHelpers.MakeAdjacencyMatrix(env,n_buses=1,skipExternals=True)
laplacian_matrix = TopologyHelpers.MakeLaplacian(env,n_buses=1,skipExternals=True)

lineOnBits = 0b11111111111111011011
adjacency_matrix_linesOff = TopologyHelpers.MakeAdjacencyMatrix(env,n_buses=1,skipExternals=True,lineOnBits=lineOnBits)
laplacian_matrix_linesOff = TopologyHelpers.MakeLaplacian(env,n_buses=1,skipExternals=True,lineOnBits=lineOnBits)

print('Adjacency matrix:')
print(adjacency_matrix)
#print(adjacency_matrix_linesOff)

# Print the laplacian matrix:
if True :
    print('\nLaplacian matrix:')
    for i in laplacian_matrix :
        tmp = ''
        for j in i :
            tmp += '{:>3}'.format(-j)
        print(tmp)

# Adjacency matrix:
if False :
    print('Adjacency matrix:')
    for i in adjacency_matrix :        
        print(' '.join(('%d'%(a) if a>=0 else '·') for a in list(i)))

#PrintAdjacencyMatrix(adjacency_matrix,skipExternals=False,nullstr='·')
print()
TopologyHelpers.PrintLineIDs(env)

print()
print('Is connected? (Manual)   :',TopologyHelpers.IsConnectedManual(adjacency_matrix))
print('Is connected? (Eigenvals):',TopologyHelpers.IsConnectedLaplacianEigenvalue(laplacian_matrix))
print('Is connected? (Manual)   :',TopologyHelpers.IsConnectedManual(adjacency_matrix_linesOff))
print('Is connected? (Eigenvals):',TopologyHelpers.IsConnectedLaplacianEigenvalue(laplacian_matrix_linesOff))

Checking All Permutations of lines on/off
-------
It is clear that many permutations of off-lines are topologically excluded because they destroy the connectedness of the grid. But how many exactly? We can determine this brute-force, as well as some properties of the excluded bitsets.

Notice below that we will represent the combination of on- and off-lines with a boolean, e.g. 0x11110 for a five-line topology with the first line disconnected.

As you can see, out of the list of all possible topologies (about 1 million), fewer than 2% (20,000) of them result in a connected grid. Furthermore, we can identify 115 distinct topologies out of 1 million that are "minimally disconnecting," e.g. they represent a unique way to disconnect the grid without any unnecessary additional disconnected lines. This can significantly reduce the amount of time required to check if a given topology results in a disconnected grid, because need only compare its bitset with only 115 other bitsets to determine whether it is disconnected.

We can also calculate a "sparsest cut" metric related to spectral partitioning and spectral bisection. The "ratio cut partition," e.g. $\frac{N_\text{lines}}{N_\text{A}\times N_\text{B}}$ where $N_\text{lines}$ is the number of disconnected lines, and $N_\text{A}$ and $N_\text{B}$ are the number of nodes in the disconnected regions. This metric shows topologies where the minimum number of cuts result in well-balanced sub-regions.

In [ ]:
%aimport CommonHelpers
%aimport TopologyHelpers

def FindConnectedAndUnconnectedBitsets(_env) :
    # Return the number of connected, unconnected bitsets, and
    # a list of the unique minimum-cut bitsets.

    histo_connected = Histo.FixedWidthConstructor(21,-0.5,20.5)
    histo_disconnected = Histo.FixedWidthConstructor(21,-0.5,20.5)
    
    n_connected = 0
    n_unconnected = 0
    minimum_cut_disconnected_bitsets = []
    connected_bitsets = []

    all_combinations = reversed(range(CommonHelpers.FullyConnectedBitset(_env.n_line) + 1))

    for i,line_bitset in enumerate(all_combinations) :

        this_nConnected = TopologyHelpers.nConnected(line_bitset,_env.n_line)
        
        # Check if another graph with fewer disconnections was already excluded, which
        # automatically would exclude this graph
        if TopologyHelpers.ExcludedByBitsetWithFewerDisconnections(line_bitset,minimum_cut_disconnected_bitsets) :
            histo_disconnected.Fill(this_nConnected)
            n_unconnected += 1
            continue

        adjacency_matrix = TopologyHelpers.MakeAdjacencyMatrix(_env,n_buses=1,
                                                               skipExternals=True,
                                                               lineOnBits=line_bitset)

        isConnected = TopologyHelpers.IsConnectedManual(adjacency_matrix)
        n_connected += isConnected
        n_unconnected += (not isConnected)

        if not isConnected :
            histo_disconnected.Fill(this_nConnected)
            TopologyHelpers.AddExcludedBitset(line_bitset,minimum_cut_disconnected_bitsets)
        if isConnected :
            histo_connected.Fill(this_nConnected)
            connected_bitsets.append(line_bitset)

    return n_connected,n_unconnected,minimum_cut_disconnected_bitsets,connected_bitsets,histo_connected,histo_disconnected

In [ ]:
n_connected,n_unconnected,minimum_cut_bitsets,connected_bitsets,histo_connected,histo_disconnected = FindConnectedAndUnconnectedBitsets(env)
#%timeit TopologyHelpers.FindConnectedAndUnconnectedBitsets()
fraction = n_connected/float(n_connected + n_unconnected)
print('Connected: {} Unconnected: {} Fraction: {:0.2f}%.'.format(n_connected,n_unconnected,100*fraction))
print('Excluded bitsets changed; new size is',len(minimum_cut_bitsets))

minimum_cut_bitsets_properties = []

for i in minimum_cut_bitsets :
    adjacency_matrix = TopologyHelpers.MakeAdjacencyMatrix(env,n_buses=1,skipExternals=True,lineOnBits=i)
    disjoint_sets = TopologyHelpers.GetDisjointSets(adjacency_matrix)
    minimum_cut_bitsets_properties.append(dict())
    set_sizes = list(len(disjoint_sets[a]) for a in disjoint_sets.keys())
    minimum_cut_bitsets_properties[-1]['bits'] = i
    minimum_cut_bitsets_properties[-1]['set_sizes'] = set_sizes
    minimum_cut_bitsets_properties[-1]['sparsest_cut_metric'] = TopologyHelpers.nDisconnected(i,20)/(set_sizes[0]*set_sizes[1])

bitsets_sorted = sorted(minimum_cut_bitsets_properties, key = lambda i: i['sparsest_cut_metric'])

ax = plt.subplot()
ax.set_yscale("log")

histo_connected.Draw()
histo_disconnected.Draw()
plt.show()

In [ ]:
print('{:<22} {:<10} {:<10} {:<8}'.format('Bitset','set sizes','nDisc. lns','sparsity'))
for ii,i in enumerate(bitsets_sorted) :
    if ii > 10 : break
    fmt = '0b{:0{}b}   ({:2d},{:2d})  {:>10} {:8.2f}'
    print(fmt.format(i['bits'],20,
                     i['set_sizes'][0],i['set_sizes'][1],
                     TopologyHelpers.nDisconnected(i['bits'],20),
                     i['sparsest_cut_metric']))
print('...')

A bitwise representation for Bus configurations, and Imposing Bus Rules
-------
In this next segment, we have several goals:
 - We want to figure out how many bus configurations are possible, given some number of lines.
 - We need to define a way to label a bus configuration in a way that is unique.

To facilitate this, we represent a bus configuration with a bitset, e.g. 0x1010 for lines 0 and 2 connected to "bus 0", and lines 1 and 3 connected to "bus 1". A bitwise representation is useful because we can use efficient bitwise operations to determine bus configuration validity.

Then we can impose rules:
 - By convention, we require the last bit to be connected to bus 1. This avoids double-counting cases with "0<-->1" symmetry.
 - We consider illegal the case where a single line is connected to a bus.
 - When a line is disconnected, the number of possible bus configurations for an "N-line bus" should reduce to the number of possible configurations for an "N-minus-one-line bus". To avoid double-counting identical bitsets in these cases, valid bitsets must have the disconnected line(s) assigned to "bus 1". In reality, of course, they are not connected at all, but we explicitly want to "invalidate" the bitsets with the disconnected line(s) assigned to "bus 0" to make the counting work out.
 
We implement these rules with functions in a module, whose results are shown below:

In [ ]:
# This helper module is mine:
import BusTopologyHelpers

for j in range(1,9) :
    print('Valid configurations in {}-line case:'.format(j),sum(list(BusTopologyHelpers.IsValidBooleanBusState(i,j) for i in range(pow(2,j)))))

items_disconnected = [1,2,3]
nEdges = 8
print('\nInvestigating case with {} lines, with these lines disconnected:'.format(nEdges),items_disconnected)
print('(Partially we want to demonstrate here that buses with')
print('{} lines and {} disconnections should have the number of options equivalent to the {}-line case)'.format(nEdges,len(items_disconnected),nEdges-len(items_disconnected)))
n = 0
for i in range(pow(2,nEdges)) :
    isValid = BusTopologyHelpers.IsValidBooleanBusState(i,nEdges,items_disconnected=items_disconnected)
    if not isValid :
        continue
    print('Flag {:0{}b} (0x{:02x}) is {}'.format(i,nEdges,i,'legal' if isValid else 'illegal'))
    n += 1
print('For {} edges, there are {} configurations that are legal,'.format(nEdges,n),'given these disconnected items:',items_disconnected)

Let us also check to make sure that our machinery correctly excludes cases where loads and generators ("externals") are not connected to the grid. The case where the external is not connected to any other line is already excluded; however, there is the additional case where 2+ externals are not connected to any lines, only to each other (and this is illegal).

In [ ]:
bus_topo = 0b11001
bus_topo_str = '{:0{}b}'.format(bus_topo,5)

i_gens,i_loads,i_disconnected = [1],[2],[0]
valid = BusTopologyHelpers.IsValidBooleanBusState(bus_topo,5,i_gens=i_gens,i_loads=i_loads,items_disconnected=i_disconnected)
valid_str = 'valid' if valid else 'NOT valid'
print('{} is {} with externals on:'.format(bus_topo_str,valid_str),i_gens + i_loads,
      'and disconnected lines:',i_disconnected)

i_gens,i_loads,i_disconnected = [1],[3],[0]
valid = BusTopologyHelpers.IsValidBooleanBusState(bus_topo,5,i_gens=i_gens,i_loads=i_loads,items_disconnected=i_disconnected)
valid_str = 'valid    ' if valid else 'NOT valid'
print('{} is {} with externals on:'.format(bus_topo_str,valid_str),i_gens + i_loads,
      'and disconnected lines:',i_disconnected)

#BusTopologyHelpers.IsValidBooleanBusState(bus_topo,5,i_gens=i_gens,i_loads=i_loads,items_disconnected=i_disconnected)

Combining Bus Configuration and Line Disconnections
-------

Let's consider roughly how many different configurations exist for "case 14" with 14 buses and 20 lines, starting from very naive assumptions and then calculating explicitly the allowed configurations under certain symmetries or rules.

For the naive expectation of line connections, we consider whether a line is connected or disconnected, which leads to $2^{n_\text{lines}}$ as the naive assumption. For a bus, we consider the fact that each element (external, or line edge) can be connected to either bus 1 or bus 2, and account for $1\leftrightarrow2$ symmetry, leading to $2^{n_\text{ext}-1}$. The element count for case 14 is as follows: 1 bus with 2 elements; 6 buses w/3 elements; 2 buses w/4 elements; 2 buses w/5 elements.

One thing we have not yet considered: in cases with one or more line disconnections, it is often the case that the number of possible bus configurations is reduced. For instance, for a bus with 6 incoming elements (and therefore normally 26 different configurations), disconnecting one line will lead to only 5 incoming elements with only 11 possible configurations. This can reduce the number of options by a factor of 2-4, though it is important to note that buses with 2 or 3 elements basically already only have 1 possible configuration.

Finally, we must consider the fact that the grid may no longer be connected when you consider both the line and bus configuration. This requires constructing the adjacency matrix, considering 2 buses per substation.

The number of bus configuration options can be computed by-hand for the nearly 20,000 line configurations, to see how many options are truly possible.

|                   | Line connections only | Bus configurations only | Lines + buses (cumulative) |
| ------            | -----                 | -----                   | ---- |
| Naive expectation | $2^{20}$ = 1,048,575  | $\prod_{i} 2^{n_i-1}$ = $4.4\times10^{12}$ | $4.6\times10^{18}$ |
| Considering lines only, exclude topo-disconnected grids (1) | 19,904 (1.9%) | ($4.4\times10^{12}$) | $8.8\times10^{16}$ |
| impose bus symmetry rules (2) | (19,904) | 23.6 million (0.0005%) | $4.7\times10^{11}$ |
| (1) + (2) + consider reduced bus options (3) | -- | -- | $1.4\times10^{9}$ (0.31%) |
| (1) + (2) + (3) + exclude topo-disconnected grids (lines+buses) (4) | -- | -- | $X.X\times10^{XX}$ |
| Valid line + bus combinations | -- | -- | $X.X\times10^{XX}$ |

In [ ]:
%aimport CommonHelpers
%aimport BusTopologyHelpers
%aimport Substation
%aimport TopologyHelpers
%autoreload

(2) Check the number of available bus configurations, assuming all lines are connected.
------------

In [ ]:
# Check the number of available bus configurations, assuming all lines are connected.

sub_classes = Substation.BuildSubstations(env)
#sub_classes[1].currentBusConfig = 0b101000
nValid = []
nValid_combi = 1

for sub in sub_classes :

    nValid.append(0)
    lineOnBits = 0b11111111111111111111

    all_combinations = reversed(range(CommonHelpers.FullyConnectedBitset(sub.nElements)+1))

    for i,sub_bitset in enumerate(all_combinations) :
        sub.SetBusConfig(sub_bitset)
        valid = sub.IsValidBooleanBusState(lineOnBits=lineOnBits)
        #validStr = 'valid' if valid else 'invalid'
        #print('Substation {:02d}: 0b{:{}b} {}'.format(sub.index,sub.currentBusConfig,sub.nElements,validStr),sub.elementIDs)
        if valid :
            nValid[-1] += 1

    nValid_combi *= nValid[-1]

print('Total number of valid combinations: {}; substation breakdown:'.format(nValid_combi),nValid)

(3) Consider reduced substation bus options (due to line disconnections)
--------

In [ ]:
def getNumberOfLineAndBusCombinations(_line_bitsets,_sub_classes) :

    nValidOverall = 0
    
    for i,line_bitset in enumerate(_line_bitsets):    

        if not i%1000 :
            print('Checking bitset {} of {}'.format(i,len(_line_bitsets)))

        nValid_subCombi = 1

        for sub in _sub_classes :

            nValid_sub = 0
            all_combinations = reversed(range(CommonHelpers.FullyConnectedBitset(sub.nElements)+1))

            for sub_bitset in all_combinations :
                sub.SetBusConfig(sub_bitset)
                valid = sub.IsValidBooleanBusState(lineOnBits=line_bitset)
                #validStr = 'valid' if valid else 'invalid'
                #print('Substation {:02d}: 0b{:{}b} {}'.format(sub.index,sub.currentBusConfig,sub.nElements,validStr),sub.elementIDs)
                if valid :
                    nValid_sub += 1

            nValid_subCombi *= nValid_sub

        nValidOverall += nValid_subCombi
    return nValidOverall

#%timeit getNumberOfLineAndBusCombinations(connected_bitsets,sub_classes)
nValid = getNumberOfLineAndBusCombinations(connected_bitsets,sub_classes)
print('Total number of valid combinations: {}'.format(nValid))

In [ ]:
%load_ext autoreload
%aimport BusTopologyHelpers
%aimport Substation
%aimport TopologyHelpers
%autoreload

sub_classes = Substation.BuildSubstations(env)
#sub_classes[1].currentBusConfig = 0b101000
for sub in sub_classes :
    valid = 'valid' if sub.IsValidBooleanBusState(lineOnBits=0b11111111111111111100) else 'invalid'
    print('Substation {:02d}: 0b{:{}b} {}'.format(sub.index,sub.currentBusConfig,sub.nElements,valid),sub.elementIDs)

adjacency_matrix_class = TopologyHelpers.AdjacencyMatrixClass(env)

# Print Adjacency matrix:
if False :
    print('Adjacency matrix:')
    for i in adjacency_matrix_class.adjacency_matrix :        
        print(' '.join(('%d'%(a) if a>=0.5 else '·') for a in list(i)))


In [ ]:
# if True :
#     n = 0
#     for i in range(32) :
#         isValid = IsValidBooleanBusState(i,5)
#         if not isValid :
#             continue
#         keptForReduced = ''
#         if (i & 0b1) :
#             keptForReduced = ' (Kept in case of last line disconnect)'
#         print('Flag {:0{}b} (0x{:02x}) is {}{}'.format(i,5,i,'legal' if isValid else 'illegal',keptForReduced))
#         n += 1
#     print('There are {} configurations that are legal'.format(n))

# if False :
#     n = 0
#     for i in range(64) :
#         isValid = IsValidBooleanBusState(i,6)
#         if not isValid :
#             continue
#         print('Flag {:0{}b} (0x{:02x}) is {}'.format(i,6,i,'legal' if isValid else 'illegal'))
#         n += 1
#     print('There are {} configurations that are legal'.format(n))

Working with the Action Space
---------

In [ ]:
tmp = env.action_space.get_change_line_status_vect()
tmp[0] = True
print('Change line status vector:',tmp)
tmp1 = env.action_space.get_set_line_status_vect()
#tmp1[3] = 1
print('Set line status vector:',tmp1)
this_first_act = env.action_space({"set_line_status":tmp1, "change_line_status":tmp})
print(this_first_act)
this_first_act.is_ambiguous()

In [ ]:
obs_after, reward, done, info = env.step(this_first_act)
print(info)
plot_helper = PlotMatplot(env.observation_space)
_ = plot_helper.plot_obs(obs_after,)

In [ ]:
_ = plot_helper.plot_info(line_values=[el for el in range(env.n_line)])

Turn off all power lines:
------

In [ ]:
env.reset()
tmp = env.action_space.get_set_line_status_vect()
tmp[0] = -1
print(tmp)
act = env.action_space({"set_line_status":tmp})
obs_after, reward, done, info = env.step(act)
print(info)
#_ = plot_helper.plot_obs(obs_after)

tmp = env.action_space.get_set_line_status_vect()
tmp[1] = -1
print(tmp)
act = env.action_space({"set_line_status":tmp})
obs_after, reward, done, info = env.step(act)
print(info)
#_ = plot_helper.plot_obs(obs_after)

#for i in range(len(env.action_space.get_set_line_status_vect())) :
#    tmp = env.action_space.get_set_line_status_vect()
#    tmp[i] = -1
#    print(tmp)
#    act = env.action_space({"set_line_status":tmp})
#    obs_after, reward, done, info = env.step(act)
#    print(info)
#    if done :
#        break    
#
#_ = plot_helper.plot_obs(obs_after)

#_ = plot_helper.plot_info(line_values=tmp)

In [ ]:
env.reset()
tmp = env.action_space.get_set_line_status_vect()
tmp[0] = -1
_ = plot_helper.plot_info()

In [ ]:
env.line_or_to_subid

In [ ]:
dir(env)

In [ ]:
env.reset()
tmp = env.action_space.get_set_line_status_vect()
print(tmp)
act = env.action_space({"set_line_status":tmp})
obs, reward, done, info = env.step(act)
for i in obs.bus_connectivity_matrix() :
    tmp = ''
    for j in i :
        tmp += ('%0d'%(int(j)) if int(j) else ' ')
    print(tmp)

In [ ]:
def isConnected(connectivity_matrix,sub_id_start=0) :
    # Checks if the topology is fully connected or not.
    # Starting point substation is where you want to start.
    element_already_visited = [sub_id_start]

Bus changes in Action Space
-----------

In [ ]:
env.reset()
set_bus_load_0 = env.action_space({"set_bus": {"loads_id": [(0,2)],"lines_or_id": [(3,2)]}})

print(set_bus_load_0)
obs_after, reward, done, info = env.step(set_bus_load_0)
print(info)
_ = plot_helper.plot_obs(obs_after)